<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-of-different-data-sources-into-dataframe" data-toc-modified-id="Import-of-different-data-sources-into-dataframe-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import of different data sources into dataframe</a></span><ul class="toc-item"><li><span><a href="#weather-data" data-toc-modified-id="weather-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>weather data</a></span></li><li><span><a href="#traffic-data" data-toc-modified-id="traffic-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>traffic data</a></span></li><li><span><a href="#Merge-MIV-and-Weather-Dataset" data-toc-modified-id="Merge-MIV-and-Weather-Dataset-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Merge MIV and Weather Dataset</a></span></li><li><span><a href="#oil-price" data-toc-modified-id="oil-price-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>oil price</a></span></li></ul></li><li><span><a href="#Load-cleaned-data-to-DWH" data-toc-modified-id="Load-cleaned-data-to-DWH-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load cleaned data to DWH</a></span><ul class="toc-item"><li><span><a href="#Load-cleaned-oilprice-to-dwh" data-toc-modified-id="Load-cleaned-oilprice-to-dwh-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load cleaned oilprice to dwh</a></span></li><li><span><a href="#Load-cleaned-weather_traffic-to-dwh" data-toc-modified-id="Load-cleaned-weather_traffic-to-dwh-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load cleaned weather_traffic to dwh</a></span></li></ul></li></ul></div>

In [3]:
import requests
import json
import os
import pandas as pd
from sqlalchemy import create_engine, text, inspect, types
from datetime import date


# DB credentials
ENDPOINT = 'trunks.cwh7dj4q8zyj.us-east-1.rds.amazonaws.com'
DB_NAME = 'trunks'
USERNAME = 'trunks'
PASSWORD = 'Corn5!!!'

# DB table name
table_name = 'weather_hourly'

# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'



print('Starting process')

#Extract
db = create_engine(conn_string)
dbconnect = db.connect()

df = pd.read_sql_table('weather_hourly', dbconnect)

#Transform
df = df[['Datum','Standort','Parameter','Einheit','Wert']]

parameter_keep = ["RainDur","T"]
df = df[df['Parameter'].isin(parameter_keep)]

df['Standort'] = df['Standort'].str.replace('Zch_','',regex=True)
df['Wert'] = df['Wert'].astype(float)
df['Datum'] = pd.to_datetime(df['Datum'],utc=False)

df = df[df["Datum"].dt.strftime('%Y') >= '2012'] #only from 2013 upwards relevant

df.head()

Starting process


,Datum,Standort,Parameter,Einheit,Wert
479995,2021-01-01 00:00:00+01:00,Stampfenbachstrasse,T,°C,2.35
479998,2021-01-01 00:00:00+01:00,Stampfenbachstrasse,RainDur,min,0.00
480003,2021-01-01 00:00:00+01:00,Schimmelstrasse,T,°C,2.77
480006,2021-01-01 00:00:00+01:00,Schimmelstrasse,RainDur,min,0.00
480010,2021-01-01 00:00:00+01:00,Rosengartenstrasse,T,°C,2.13


In [4]:
df.sort_values("Datum")

,Datum,Standort,Parameter,Einheit,Wert
2105601,2012-01-01 00:00:00+01:00,Stampfenbachstrasse,RainDur,min,60.00
2105596,2012-01-01 00:00:00+01:00,Schimmelstrasse,T,°C,7.92
2105602,2012-01-01 00:00:00+01:00,Stampfenbachstrasse,T,°C,7.25
2105595,2012-01-01 00:00:00+01:00,Schimmelstrasse,RainDur,min,6.27
2105609,2012-01-01 01:00:00+01:00,Schimmelstrasse,RainDur,min,0.12
...,...,...,...,...,...
1274748,2022-04-28 18:00:00+01:00,Stampfenbachstrasse,RainDur,min,0.00
1274753,2022-04-28 18:00:00+01:00,Schimmelstrasse,T,°C,18.26
1274756,2022-04-28 18:00:00+01:00,Schimmelstrasse,RainDur,min,0.00
1274760,2022-04-28 18:00:00+01:00,Rosengartenstrasse,T,°C,17.84


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512040 entries, 479995 to 2228563
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype                               
---  ------     --------------   -----                               
 0   Datum      512040 non-null  datetime64[ns, pytz.FixedOffset(60)]
 1   Standort   512040 non-null  object                              
 2   Parameter  512040 non-null  object                              
 3   Einheit    512040 non-null  object                              
 4   Wert       508384 non-null  float64                             
dtypes: datetime64[ns, pytz.FixedOffset(60)](1), float64(1), object(3)
memory usage: 23.4+ MB
